# 🎓 Learning Arc: Module 12 - Usage Metering & Analytics

## 📋 Purpose
Learn to build production-grade usage metering systems for multi-tenant SaaS applications. Bridge the gap between having a multi-tenant RAG API and being able to bill customers accurately or detect quota overages.

## 🔑 Concepts Covered
1. **Event-driven metering** - Capture usage events with tenant attribution
2. **OLAP databases** - ClickHouse for 10-100x better aggregation performance
3. **Materialized views** - Real-time pre-computed aggregations (hourly/daily)
4. **Async buffering** - Non-blocking writes with <5ms overhead per request
5. **Graceful degradation** - Fallback to local storage when database unavailable
6. **Cost calculation** - Map usage quantities to billable amounts
7. **Quota enforcement** - Prevent runaway costs with real-time overage detection
8. **Billing export** - Generate monthly invoices from immutable event logs

## ✅ After Completing This Module
You will be able to:
- Design and implement a scalable usage metering pipeline
- Choose between PostgreSQL and ClickHouse for time-series data
- Build non-blocking event trackers that don't slow down APIs
- Calculate costs from usage events with configurable pricing
- Enforce tenant quotas and detect overages in real-time
- Export billing data for invoice generation

## 🗺️ Context in Track L3.M12
This module is part of **Level 3: SaaS Operations & Monetization**:
- **Prerequisite**: L2.M7 (Observability) + L2.M11 (Multi-Tenant Architecture)
- **Current Module**: M12.1 - Usage Metering & Analytics
- **Next Module**: M12.2 - Advanced Analytics & Dashboarding with Grafana
- **Related Modules**: M12.3 (Self-Service Tenant Onboarding), M12.4 (Tenant Lifecycle Management)

---

# Module 12.1: Usage Metering & Analytics

**Production-grade usage metering for multi-tenant SaaS applications**

## Overview

This module teaches building usage metering systems that bridge the gap between having a multi-tenant RAG API and being able to bill customers accurately or detect quota overages.

**Key Architecture Components:**
- Event capture at API endpoints with tenant attribution
- Append-only storage in ClickHouse (columnar OLAP database)
- Materialized views for real-time aggregations (hourly, daily)
- Per-tenant dashboards and quota enforcement
- Billing export for monthly invoice generation

**Why ClickHouse over PostgreSQL:**
- 10-100x better query performance on aggregations
- Columnar storage optimized for time-series data
- Handles 10,000+ events/day per tenant efficiently

**Critical Design Decisions:**
- Async architecture maintains <5ms overhead per request
- Immutable event log provides complete audit trail
- Fallback to local files if ClickHouse unavailable

In [ ]:
# Setup and imports
import os
import sys
import json
from datetime import datetime, timedelta

# OFFLINE mode for L3 consistency
OFFLINE = os.getenv("OFFLINE", "false").lower() == "true"
if OFFLINE:
    print("⚠️ Running in OFFLINE mode — ClickHouse/DB calls will be skipped (mocked or use fallback).")

# Import our core module
from src.l3_m12_usage_metering_analytics import (
    UsageEvent, TenantQuota, ClickHouseSchema,
    UsageTracker, CostCalculator, QuotaManager, BillingExporter
)
from config import get_clickhouse_client, get_config

# Load sample data (from parent directory)
with open('../example_data.json') as f:
    sample_data = json.load(f)

print("✓ Imports successful")
print(f"✓ Loaded {len(sample_data['sample_events'])} sample events")

# Expected:
# ✓ Imports successful
# ✓ Loaded 5 sample events

## Section 2: ClickHouse Schema Setup

**Four Primary Structures:**

1. **`usage_events`** - Append-only event log
   - Partitioned monthly for efficient queries
   - Ordered by tenant_id + timestamp
   - 36-month TTL for compliance

2. **`usage_hourly`** - Materialized view
   - Real-time hourly aggregations
   - Tracks quantity and costs per event type

3. **`usage_daily`** - Daily summaries
   - Per-tenant daily totals
   - Queries, tokens, storage metrics

4. **`tenant_quotas`** - Quota tracking
   - Limits and consumption per tenant

In [ ]:
# Initialize ClickHouse client (with graceful fallback)
client = get_clickhouse_client()

if client:
    # Initialize schema
    success = ClickHouseSchema.initialize_schema(client)
    print(f"✓ Schema initialized: {success}")
    
    # Show schema SQL
    print("\n📋 Schema components:")
    for i, sql in enumerate(ClickHouseSchema.get_schema_sql(), 1):
        table_name = sql.split("TABLE")[1].split("(")[0].strip().replace("IF NOT EXISTS", "").strip()
        print(f"  {i}. {table_name}")
else:
    print("⚠️ Skipping schema init (ClickHouse not available)")

# Expected:
# ⚠️ Skipping schema init (ClickHouse not available)
# OR
# ✓ Schema initialized: True
# 📋 Schema components: usage_events, usage_hourly, usage_daily, tenant_quotas

## Section 3: Usage Tracker with Async Buffering

**Design Goals:**
- **Non-blocking writes** - Cannot slow down API responses
- **<5ms overhead** per request
- **Batch insertion** - 100 events or 1-second intervals
- **Fallback storage** - Local JSONL files if ClickHouse unavailable

**Key Feature: Graceful Degradation**
If the database is down, events are written to local files for later replay.

In [ ]:
# Create sample events from example data
import asyncio
import uuid

# Initialize tracker
tracker = UsageTracker(client)

# Create UsageEvent objects from sample data
events = []
for evt_data in sample_data['sample_events'][:3]:  # First 3 events
    event = UsageEvent(
        event_id=evt_data['event_id'],
        tenant_id=evt_data['tenant_id'],
        event_type=evt_data['event_type'],
        quantity=evt_data['quantity'],
        timestamp=datetime.fromisoformat(evt_data['timestamp'].replace('Z', '+00:00')),
        metadata=evt_data['metadata']
    )
    events.append(event)
    print(f"📝 Event: {event.tenant_id} - {event.event_type} ({event.quantity})")

print(f"\n✓ Created {len(events)} events")

# Expected:
# 📝 Event: tenant_acme - query (1)
# 📝 Event: tenant_acme - token_input (1500)
# 📝 Event: tenant_acme - token_output (800)
# ✓ Created 3 events

## Section 4: Cost Calculation Logic

**Pricing Model:**
Maps usage quantities to billable amounts based on configurable pricing:

- **Queries**: $0.01 per query
- **Input Tokens**: $0.003 per 1K tokens
- **Output Tokens**: $0.015 per 1K tokens (5x input cost)
- **Storage**: $0.10 per GB

**Design:** Centralized pricing configuration allows easy updates without code changes.

In [ ]:
# Calculate costs for sample events
calculator = CostCalculator()

print("💰 Cost Calculation:")
total_cost = 0.0
for event in events:
    cost = calculator.calculate_event_cost(event)
    total_cost += cost
    print(f"  {event.event_type:15s} x {event.quantity:6.0f} = ${cost:.4f}")

print(f"\n✓ Total cost: ${total_cost:.4f}")

# Verify against expected costs
expected = sample_data['expected_costs']
print(f"✓ Expected: ${expected['evt_001'] + expected['evt_002'] + expected['evt_003']:.4f}")

# Expected:
# 💰 Cost Calculation:
#   query           x      1 = $0.0100
#   token_input     x   1500 = $0.0045
#   token_output    x    800 = $0.0120
# ✓ Total cost: $0.0265

## Section 5: Quota Management & Overage Detection

**Purpose:** Prevent runaway costs and enforce fair-use policies

**Key Capabilities:**
- Set per-tenant limits (queries/day, tokens/day, storage GB)
- Real-time quota checking
- Overage detection with configurable enforcement
- Support for overage-allowed vs. hard-limit modes

**Critical for:** Multi-tenant SaaS with freemium or tiered pricing

In [ ]:
# Set quota for sample tenant
if client:
    quota_mgr = QuotaManager(client)
    
    # Create quota from sample data
    quota = TenantQuota(
        tenant_id="tenant_acme",
        queries_per_day=1000,
        tokens_per_day=100000,
        storage_gb=10.0,
        overage_allowed=True
    )
    
    success = quota_mgr.set_quota(quota)
    print(f"✓ Quota set: {success}")
    
    # Check quota status
    status = quota_mgr.check_quota("tenant_acme")
    print(f"\n📊 Quota Status:")
    print(f"  Status: {status.get('status', 'unknown')}")
    print(f"  Remaining queries: {status.get('remaining', {}).get('queries', 'N/A')}")
else:
    print("⚠️ Skipping quota operations (ClickHouse not available)")

# Expected:
# ⚠️ Skipping quota operations (ClickHouse not available)
# OR
# ✓ Quota set: True
# 📊 Quota Status: within_quota

## Section 6: Billing Export & Invoice Generation

**Monthly Invoice Data Structure:**
- Period (year-month)
- Line items with quantity, unit price, and amount
- Total amount in USD
- Detailed breakdown by event type

**Integration:** Exports can feed into Stripe, QuickBooks, or custom billing systems.

**Audit Trail:** Immutable event log ensures invoices can always be regenerated and verified.

In [ ]:
# Generate monthly invoice
if client:
    exporter = BillingExporter(client)
    
    # Generate invoice for November 2025
    invoice = exporter.export_monthly_invoice("tenant_acme", 2025, 11)
    
    if 'error' not in invoice:
        print(f"📄 Invoice: {invoice['period']}")
        print(f"  Tenant: {invoice['tenant_id']}")
        print(f"\n  Line Items:")
        for item in invoice.get('line_items', [])[:3]:  # Show first 3
            print(f"    {item['description']:15s} {item['quantity']:6.0f} {item['unit']:10s} @ ${item['unit_price']:.4f} = ${item['amount']:.2f}")
        print(f"\n  💵 Total: ${invoice['total_amount']:.2f}")
    else:
        print(f"⚠️ Invoice generation skipped: {invoice['error']}")
else:
    print("⚠️ Skipping invoice generation (ClickHouse not available)")

# Expected:
# ⚠️ Skipping invoice generation (ClickHouse not available)
# OR
# 📄 Invoice: 2025-11
#   Line Items: [...list of charges...]
#   💵 Total: $X.XX

## Section 7: Common Failure Modes & Fixes

### 1. Metering Data Loss
**Risk:** Events not written if ClickHouse unavailable  
**Solution:** Fallback to local file buffering with replay mechanism

### 2. Aggregation Bugs
**Risk:** Discrepancies between raw events and summaries  
**Solution:** Regular reconciliation queries comparing totals

### 3. Dashboard Performance
**Risk:** Slow queries for large tenants with years of history  
**Solution:** Monthly partitioning + pre-aggregated views (hourly/daily)

### 4. Overage Detection Delays
**Risk:** Tenants hit quotas without real-time notification  
**Solution:** Sub-minute aggregation windows with alert rules

### 5. Billing Reconciliation
**Risk:** Invoiced amounts don't match usage records  
**Solution:** Immutable event log enables auditing back to source

In [ ]:
# Demonstrate fallback behavior (Data Loss Mitigation)
import os

# Tracker with no client will use fallback
fallback_tracker = UsageTracker(client=None)

# Create test event
test_event = UsageEvent(
    event_id="test_001",
    tenant_id="tenant_test",
    event_type="query",
    quantity=1,
    timestamp=datetime.now(),
    metadata={"test": True}
)

print("🔄 Testing fallback storage...")
# Note: In real async code, use await
# For demo, we'll show the concept
print(f"✓ Fallback file path: {fallback_tracker.client is None}")
print(f"✓ Event will write to: {os.path.basename(get_config()['fallback_file_path']) if 'fallback_file_path' in get_config() else 'usage_events_fallback.jsonl'}")

# Expected:
# 🔄 Testing fallback storage...
# ✓ Fallback file path: True
# ✓ Event will write to: usage_events_fallback.jsonl

## Section 8: Decision Card - When to Use This System

### ✅ USE THIS APPROACH WHEN:

1. **50+ customers** requiring billing transparency
2. **Significant per-tenant usage variation** (10x+ differences)
3. **Usage-based pricing** is core to business model
4. **10,000+ events/day** across all tenants
5. **Audit requirements** demand complete event history

### ❌ DO NOT USE WHEN:

1. **Early-Stage MVP (<50 customers)**
   - Alternative: Manual tracking or simple database queries
   - Reason: Infrastructure overhead unjustified

2. **Low-Volume Internal Tools (<100 queries/day)**
   - Alternative: Flat-rate or seat-based pricing
   - Reason: Billing complexity premature

3. **High-Frequency Trading (>10K requests/second)**
   - Alternative: Specialized stream processing (Kafka + Flink)
   - Reason: Event volume overwhelms typical metering

### 🎯 Quick Decision Test:
**Use this system if:** 50+ tenants × variable usage × need billing transparency = YES

### 💰 Production Costs:
- **Up to 50 tenants / 50K events/day:** Single ClickHouse instance (~$50/month)
- **Estimated monthly cost:** $185 (ClickHouse) + operational overhead
- **Engineering effort:** 4-8 hours implementation + ongoing monitoring

In [ ]:
# Quick reference: Configuration summary
config = get_config()

print("📊 System Configuration Summary:")
print(f"\n  ClickHouse: {config['clickhouse']['host']}:{config['clickhouse']['port']}")
print(f"  Database: {config['clickhouse']['database']}")
print(f"\n  Buffering:")
print(f"    Buffer size: {config['buffering']['buffer_size']} events")
print(f"    Flush interval: {config['buffering']['flush_interval']}s (<5ms overhead)")
print(f"\n  Pricing:")
for key, value in config['pricing'].items():
    print(f"    {key}: ${value}")
print(f"\n  Retention: {config['retention_months']} months (compliance)")

# Expected:
# 📊 System Configuration Summary:
#   ClickHouse: localhost:9000
#   Buffer size: 100 events
#   Flush interval: 1.0s (<5ms overhead)
#   Pricing: {...}
#   Retention: 36 months

## Section 9: Summary & Next Steps

### ✅ What We Built:

1. **ClickHouse Schema** - 4-table architecture with materialized views
2. **Usage Tracker** - Async buffering with <5ms overhead
3. **Cost Calculator** - Configurable pricing engine
4. **Quota Manager** - Real-time overage detection
5. **Billing Exporter** - Monthly invoice generation

### 🔑 Key Takeaways:

- **Performance:** 10-100x better than PostgreSQL for aggregations
- **Reliability:** Fallback storage prevents data loss
- **Auditability:** Immutable event log for compliance
- **Scalability:** Handles 10,000+ events/day per tenant

### 📈 Production Readiness:

- **Metrics to Monitor:** Event write latency, buffer flush rate, aggregation lag
- **Cost:** $185/month for 50 tenants
- **Effort:** 4-8 hours implementation

### 🚀 Next Module:

**Module 12.2:** Advanced analytics and dashboarding with Grafana

---

**Practathon Challenges:**
- **Easy (60-90 min):** Implement basic query metering
- **Medium (2-3 hours):** Add token tracking + Grafana dashboard
- **Hard (5-6 hours):** Full integration with overage alerts